# Calculating SOAP descriptors for zeolites

In this notebook, we will calculate the SOAP descriptors for IZA zeolites using the package `dscribe`.

In [1]:
import os
import warnings
import pandas as pd
import numpy as np
import amd
from ase.io import read
from dscribe.descriptors import SOAP

warnings.filterwarnings("ignore")

## Calculation of the SOAP descriptors

In [2]:
species = ["Si", "O"]
r_cut = 6.0
n_max = 8
l_max = 6

# Setting up the SOAP descriptor
soap = SOAP(
    species=species,
    periodic=True,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner",
)

In [10]:
path = os.path.abspath('../dsets/iza_experimental')
files = sorted([f for f in os.listdir(path) if f.endswith(".cif")])

## Timing of SOAP vectors + I/O by ASE

In [12]:
%%timeit

soap_vecs = []
for f in files:
    atoms = read(os.path.join(path, f))
    soap_vecs.append(soap.create(atoms))
    
soap_vecs = np.array(soap_vecs)

dotprod = (soap_vecs @ soap_vecs.T)
norms = np.diag(dotprod)
kernel = dotprod / np.sqrt(norms.reshape(-1, 1) * norms.reshape(1, -1))
dm = np.sqrt(2 - 2 * kernel)

19.1 s ± 387 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Timing of creating SOAP vectors without the I/O from ASE

In [14]:
dset = [
    read(os.path.join(path, f))
    for f in files
]

In [15]:
%%timeit

soap_vecs = []
for atoms in dset:
    soap_vecs.append(soap.create(atoms))
    
soap_vecs = np.array(soap_vecs)

dotprod = (soap_vecs @ soap_vecs.T)
norms = np.diag(dotprod)
kernel = dotprod / np.sqrt(norms.reshape(-1, 1) * norms.reshape(1, -1))
dm = np.sqrt(2 - 2 * kernel)

5.79 s ± 36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Calculation of AMD

In [13]:
%%timeit

k = 100
path = os.path.abspath('../dsets/iza_experimental')
_dm = amd.compare(path, by='AMD', k=k)

1.81 s ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
